<center> <img src = https://raw.githubusercontent.com/AndreyRysistov/DatasetsForPandas/main/hh%20label.jpg alt="drawing" style="width:400px;">

# <center> Проект: Анализ вакансий из HeadHunter
   

In [69]:
# Импортируем необходимые для работы библиотеки:
import pandas as pd
import psycopg2

In [70]:
# Задаем параметры для подключения к базе в PostgreSQL:
# ***Здесь были параметры, но их удалили перед загрузкой на Git Hub**
DBNAME = 
USER = 
PASSWORD = 
HOST = 
PORT = 

In [71]:
# Создаём соединение с заданными параметрами:
connection = psycopg2.connect(
    dbname=DBNAME,
    user=USER,
    host=HOST,
    password=PASSWORD,
    port=PORT
)

# Юнит 3. Предварительный анализ данных

1. Напишите запрос, который посчитает количество вакансий в нашей базе (вакансии находятся в таблице vacancies). 

In [72]:
# Код запроса:
query_3_1 = f'''SELECT COUNT(*)
                FROM public.vacancies
            '''

In [73]:
# Выполним запрос и выведем результат - количество вакансий в базе:
df_query_3_1 = pd.read_sql_query(query_3_1, connection)
df_query_3_1

C:\Users\anupa\AppData\Local\Temp\ipykernel_21092\4061902056.py:2: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



,count
0,49197


2. Напишите запрос, который посчитает количество работодателей (таблица employers). 

In [74]:
# Код запроса:
query_3_2 = f'''SELECT COUNT(*)
                FROM public.employers
            '''

In [75]:
# Выполним запрос и выведем результат - количество работодателей:
df_query_3_2  = pd.read_sql_query(query_3_2, connection)
df_query_3_2

C:\Users\anupa\AppData\Local\Temp\ipykernel_21092\743110899.py:2: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



,count
0,23501


3. Посчитате с помощью запроса количество регионов (таблица areas).

In [76]:
# Код запроса:
query_3_3 = f'''SELECT COUNT(*)
                FROM public.areas
            '''

In [77]:
# Выполним запрос и выведем результат - количество регионов:
df_query_3_3 = pd.read_sql_query(query_3_3, connection)
df_query_3_3

C:\Users\anupa\AppData\Local\Temp\ipykernel_21092\4148501598.py:2: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



,count
0,1362


4. Посчитате с помощью запроса количество сфер деятельности в базе (таблица industries).

In [78]:
# Код запроса:
query_3_4 = f'''SELECT COUNT(*)
                FROM public.industries
            '''

In [79]:
# Выполним запрос и выведем результат - количество сфер деятельности:
df_query_3_4 = pd.read_sql_query(query_3_4, connection)
df_query_3_4

C:\Users\anupa\AppData\Local\Temp\ipykernel_21092\4103242369.py:2: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



,count
0,294


***

#### Выводы по предварительному анализу данных
* База данных вакансий HeadHunter насчитывает около 50 тыс. вакансий, что является более чем достаточным для анализа и получения репрезентативных результатов.
* Количество работодателей намного меньше - 23,5 тыс. Это означает, что у некоторых работодателей много открытых вакансий - скорее всего это крупные корпорации с большим штатом сотрудников.
* В таблице areas содержится информация о регионах -  код города и его название. Всего 1362 города. По состоянию на 19 февраля 2023 года в России насчитывается 1118 городов. Таким образом, в базе содержатся города и за пределами России, или, возможно поселки, села и прочие населенные пункты.
* Количество сфер деятельности в справочнике - 294. Это меньше, чем в Реестре областей и видов профессиональной деятельности Минтруда России, значит не все они востребованы и имеют вакансии на HeadHunter.


# Юнит 4. Детальный анализ вакансий

1. Напишите запрос, который позволит узнать, сколько (cnt) вакансий в каждом регионе (area).
Отсортируйте по количеству вакансий в порядке убывания.

In [80]:
# Код запроса: посчитаем количество вакансий cnt в каждом регионе и отсортируем в порядке убывания:
query_4_1 = f'''SELECT a.name AS area, COUNT(*) cnt
                FROM public.vacancies v
                LEFT JOIN public.areas a ON v.area_id = a.id
                GROUP BY a.name
                ORDER BY cnt DESC
            '''

In [81]:
# Выполним запрос и выведем результат:
df_query_4_1  = pd.read_sql_query(query_4_1, connection)
df_query_4_1

C:\Users\anupa\AppData\Local\Temp\ipykernel_21092\2180486704.py:2: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



,area,cnt
0,Москва,5333
1,Санкт-Петербург,2851
2,Минск,2112
3,Новосибирск,2006
4,Алматы,1892
...,...,...
764,Тарко-Сале,1
765,Новоаннинский,1
766,Бирск,1
767,Сасово,1


2. Напишите запрос, чтобы определить у какого количества вакансий заполнено хотя бы одно из двух полей с зарплатой.

In [82]:
# Код запроса: посчитаем количество вакансий cnt с условием, что хотя бы одно из значений salary_from и salary_to является непустым:
query_4_2 = f'''SELECT COUNT(*) cnt
                FROM public.vacancies v
                WHERE v.salary_from IS NOT NULL OR v.salary_to IS NOT NULL
            '''

In [83]:
# Выполним запрос и выведем результат:
df_query_4_2  = pd.read_sql_query(query_4_2, connection)
df_query_4_2

C:\Users\anupa\AppData\Local\Temp\ipykernel_21092\1436269751.py:2: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



,cnt
0,24073


3. Найдите средние значения для нижней и верхней границы зарплатной вилки. Округлите значения до целого.

In [84]:
# Код запроса: найдем среднее по значениям salary_from и salary_to, округлим до целого:
query_4_3 = f'''SELECT ROUND(AVG(v.salary_from)) avg_salary_from, ROUND(AVG(v.salary_to)) avg_salary_to
                FROM public.vacancies v
            '''

In [85]:
# Выполним запрос и выведем результат:
df_query_4_3  = pd.read_sql_query(query_4_3, connection)
df_query_4_3

C:\Users\anupa\AppData\Local\Temp\ipykernel_21092\1101898888.py:2: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



,avg_salary_from,avg_salary_to
0,71065.0,110537.0


4. Напишите запрос, который выведет количество вакансий для каждого сочетания типа рабочего графика (schedule) и типа трудоустройства (employment), используемого в вакансиях. Результат отсортируйте по убыванию количества.


In [86]:
# Код запроса: сгруппируем список вакансий по типу рабочего графика (schedule) и типу трудоустройства (employment), выведем 
# количество вакансии cnt и отсортируем по убыванию:
query_4_4 = f'''SELECT v.schedule, v.employment, COUNT(*) cnt
                FROM public.vacancies v
                GROUP BY v.schedule, v.employment
                ORDER BY cnt DESC
            '''

In [87]:
# Выполним запрос и выведем результат:
df_query_4_4  = pd.read_sql_query(query_4_4, connection)
df_query_4_4

C:\Users\anupa\AppData\Local\Temp\ipykernel_21092\2778541719.py:2: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



,schedule,employment,cnt
0,Полный день,Полная занятость,35367
1,Удаленная работа,Полная занятость,7802
2,Гибкий график,Полная занятость,1593
3,Удаленная работа,Частичная занятость,1312
4,Сменный график,Полная занятость,940
5,Полный день,Стажировка,569
6,Вахтовый метод,Полная занятость,367
7,Полный день,Частичная занятость,347
8,Гибкий график,Частичная занятость,312
9,Полный день,Проектная работа,141


5. Напишите запрос, выводящий значения поля Требуемый опыт работы (experience) в порядке возрастания количества вакансий, в которых указан данный вариант опыта. 

In [88]:
# Код запроса: сгруппируем список по опыту работы (experience), посчитаем количество вакансий cnt и отсортируем в порядке возрастания:
query_4_5 = f'''SELECT v.experience, COUNT(*) cnt
                FROM public.vacancies v
                GROUP BY v.experience
                ORDER BY cnt
            '''

In [89]:
# Выполним запрос и выведем результат:
df_query_4_5  = pd.read_sql_query(query_4_5, connection)
df_query_4_5

C:\Users\anupa\AppData\Local\Temp\ipykernel_21092\1501909204.py:2: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



,experience,cnt
0,Более 6 лет,1337
1,Нет опыта,7197
2,От 3 до 6 лет,14511
3,От 1 года до 3 лет,26152


***

### Выводы по детальному анализу вакансий
* Анализ распределения вакансий относительно регионов показал, что в ТОП-5 по количеству входит Москва (5333 вакансии), Санкт-Петербург (2851), Минск (2112), Новосибирск (2006), Алматы (1892). Это крупные города-миллионники, среди них столицы стран, поэтому логично, что там наиболее востребованы рабочие места. Самая большая численность населения в Москве - на 2023 год в столице проживает 13 млн человек. Также Москва - центр управления большой частью экономики страны. По состоянию на ноябрь 2019 года из 200 крупнейших предприятий страны 104 зарегистрированы в Москве. Поэтому здесь количество вакансий в 2 - 2,5 раза выше, чем в остальных городах ТОП-5. 
* Средняя нижняя граница зарплатной вилки по всем вакансиям составляет 71 065 руб., верхняя - 110 537 руб. По данным ГородРабот.ру, средняя зарплата в России за 2023 год ‒ 63 272 руб. Работодатели в среднем предлагают ЗП выше этого значения, но и не все компании из базы находятся в России. Можно сделать вывод, что уровень оплаты труда увеличивается, а разница между средним максимальным и минимальным значением составляет 40 000 руб. - что достаточно мало, если рассматривать не только рабочие, но и руководящие должности. Данный анализ явно не очень показательный, можно более подробно рассмотреть средние значения ЗП в разрезе вакансий, графиков работы, опыта и местоположения.
* Большинство вакансий (35 367 шт.) предусматривают полный рабочий день и полную занятость, что является самым распространенным графиком работы и типом трудоустройства. Немалую часть составляют вакансии с удаленной работой на полный рабочий день (7 802 шт.), что является очень популярным: во время COVID-19 многие организации перешли на удаленку и некоторые из них до сих пор предпочитают такой график работы. Также это могут быть вакансии для фрилансеров, которые работают из любой точки мира в любое время.
* Чуть больше половины всех вакансий ищут соискателей с опытом работы 1-3 года. Скорее всего, это самые низшие должности в организации, которые всегда востребованы. Интересно, что с опытом работы от 6 лет всего 1337 вакансий (около 3% от общего количества). Возможно, что чаще всего на такие должности переходят люди из тех же организаций в процессе повышения по службе, поэтому не требуется поиск сотрудника с помощью специализированных сайтов.


# Юнит 5. Анализ работодателей

1. Напишите запрос, который позволит узнать, какие работодатели находятся на первом и пятом месте по количеству вакансий.

In [90]:
# Код запроса: объединим таблицу с вакансиями (vacancies) и с работодателями (employers), чтобы извлечь наименование организации, 
# сгруппируем список по компаниям, выведем общее количество вакансий, отсортируем в порядке убывания и выведем только первые 5 строчек:
query_5_1 = f'''SELECT e.name, COUNT(v.id) cnt
                FROM public.vacancies v
                LEFT JOIN public.employers e ON v.employer_id = e.id
                GROUP BY e.name
                ORDER BY cnt DESC
                LIMIT 5
            '''

In [91]:
# Выполним запрос и выведем результат:
df_query_5_1 = pd.read_sql_query(query_5_1, connection)
df_query_5_1

C:\Users\anupa\AppData\Local\Temp\ipykernel_21092\28746455.py:2: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



,name,cnt
0,Яндекс,1933
1,Ростелеком,491
2,Тинькофф,444
3,СБЕР,428
4,Газпром нефть,331


2. Напишите запрос, который для каждого региона выведет количество работодателей и вакансий в нём.
Среди регионов, в которых нет вакансий, найдите тот, в котором наибольшее количество работодателей.


In [92]:
# Код запроса: объединим таблицу с регионами (areas) сначала с работодателями (employers), а затем с вакансиями (vacancies). Используем
# LEFT JOIN, чтобы в выборке остались все регионы из таблицы areas, а остальные данные "подтянулись" с помощью столбцов с id.
# Сгруппируем список по регионам, выберем только те, где нет вакансий (v.id IS NULL) и отсортируем по убыванию количества работодателей 
# employers_count:
query_5_2 = f'''SELECT a.name, COUNT(DISTINCT e.id) employers_count, COUNT(DISTINCT v.id) vacancies_count
                FROM public.areas a
                LEFT JOIN public.employers e ON e.area = a.id
                LEFT JOIN public.vacancies v ON e.area = v.area_id
                WHERE v.id IS NULL
                GROUP BY a.name
                ORDER BY employers_count DESC
            '''

In [93]:
# Выполним запрос и выведем результат:
df_query_5_2 = pd.read_sql_query(query_5_2, connection)
df_query_5_2

C:\Users\anupa\AppData\Local\Temp\ipykernel_21092\3546579850.py:2: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



,name,employers_count,vacancies_count
0,Россия,410,0
1,Казахстан,207,0
2,Московская область,75,0
3,Краснодарский край,19,0
4,Беларусь,18,0
...,...,...,...
889,Жабинка,0,0
890,Железногорск-Илимский,0,0
891,Жетысай,0,0
892,Жигалово,0,0


3. Для каждого работодателя посчитайте количество регионов, в которых он публикует свои вакансии. Отсортируйте результат по убыванию количества.


In [94]:
# Код запроса: объединим таблицу с регионами (vacancies) и работодателями (employers). Сгруппируем по наименованию организации и посчитаем 
# количество уникальных регионов areas_count, где работодатель публикует свои вакансии (с помощью COUNT(DISTINCT v.area_id)). Отсортируем по 
# убыванию areas_count:
query_5_3 = f'''SELECT e.name, COUNT(DISTINCT v.area_id) areas_count
                FROM public.vacancies v
                JOIN public.employers e ON v.employer_id = e.id
                GROUP BY e.name
                ORDER BY areas_count DESC
            '''

In [95]:
# Выполним запрос и выведем результат:
df_query_5_3 = pd.read_sql_query(query_5_3, connection)
df_query_5_3

C:\Users\anupa\AppData\Local\Temp\ipykernel_21092\1979331862.py:2: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



,name,areas_count
0,Яндекс,181
1,Ростелеком,152
2,Спецремонт,116
3,Поляков Денис Иванович,88
4,ООО ЕФИН,71
...,...,...
14761,UniSol,1
14762,UNISTORY LLC,1
14763,UNIT6,1
14764,United Distribution,1


4. Напишите запрос для подсчёта количества работодателей, у которых не указана сфера деятельности. 

In [96]:
# Код запроса: объединим таблицу с работодателями (employers) и сферами деятельности (employers_industries). Выведем количество  
# работодателей, где графа industry_id является пустой (ei.industry_id IS NULL):
query_5_4 = f'''SELECT COUNT(e.id) employers
                FROM public.employers e
                LEFT JOIN public.employers_industries ei ON e.id = ei.employer_id
                WHERE ei.industry_id IS NULL
            '''

In [97]:
# Выполним запрос и выведем результат:
df_query_5_4 = pd.read_sql_query(query_5_4, connection)
df_query_5_4

C:\Users\anupa\AppData\Local\Temp\ipykernel_21092\1792042116.py:2: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



,employers
0,8419


5. Напишите запрос, чтобы узнать название компании, находящейся на третьем месте в алфавитном списке (по названию) компаний, у которых указано четыре сферы деятельности. 

In [98]:
# Код запроса: объединим таблицу с работодателями (employers) и работодателями-сферами деятельности (employers_industries). 
# Выполним группировку по id работодателя и применим условие, что количество сфер деятельности равно 4 (HAVING COUNT(ei.industry_id) = 4).  
# Выведем наименование организаций в алфавитном подрядке, уберем первые 2 значения и оставим только одно - 3-е:
query_5_5 = f'''SELECT e.name
                FROM public.employers e
                JOIN public.employers_industries ei ON e.id = ei.employer_id
                GROUP BY e.id
                HAVING COUNT(ei.industry_id) = 4
                ORDER BY e.name
                OFFSET 2 LIMIT 1
            '''

In [99]:
# Выполним запрос и выведем результат:
df_query_5_5 = pd.read_sql_query(query_5_5, connection)
df_query_5_5

C:\Users\anupa\AppData\Local\Temp\ipykernel_21092\1665810364.py:2: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



,name
0,2ГИС


6. С помощью запроса выясните, у какого количества работодателей в качестве сферы деятельности указана Разработка программного обеспечения.


In [100]:
# Код запроса: объединим таблицу с работодателями-сферами деятельности (employers_industries) и справочником сфер деятельности (industries). 
# Наложим условие на наименование сферы деятельности WHERE i.name = 'Разработка программного обеспечения' и посчитаем количество 
# работодателей с таким наименованием:
query_5_6 = f'''SELECT COUNT(ei.employer_id)
                FROM public.employers_industries ei
                JOIN public.industries i ON ei.industry_id = i.id
                WHERE i.name = 'Разработка программного обеспечения'
            '''

In [101]:
# Выполним запрос и выведем результат:
df_query_5_6 = pd.read_sql_query(query_5_6, connection)
df_query_5_6

C:\Users\anupa\AppData\Local\Temp\ipykernel_21092\1722312980.py:2: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



,count
0,3553


7. Для компании «Яндекс» выведите список регионов-миллионников, в которых представлены вакансии компании, вместе с количеством вакансий в этих регионах. Также добавьте строку Total с общим количеством вакансий компании. Результат отсортируйте по возрастанию количества.

Список городов-милионников надо взять [отсюда](https://ru.wikipedia.org/wiki/%D0%93%D0%BE%D1%80%D0%BE%D0%B4%D0%B0-%D0%BC%D0%B8%D0%BB%D0%BB%D0%B8%D0%BE%D0%BD%D0%B5%D1%80%D1%8B_%D0%A0%D0%BE%D1%81%D1%81%D0%B8%D0%B8). 

Если возникнут трудности с этим задание посмотрите материалы модуля  PYTHON-17. Как получать данные из веб-источников и API. 

In [102]:
# Извлечем список городов-миллионников из страницы Википедии и добавим в новый датафрейм df. 
# Далее создадим кортеж с помощью метода tuple(), чтобы обращаться к нему с помощью SQL-запросов:
import urllib.parse as req
path_common = req.urlparse('https://ru.wikipedia.org/wiki/Города-миллионеры_России#Список_городов-миллионеров')
url = path_common.scheme + '://' + req.quote(path_common.netloc) +  req.quote(path_common.path) + '#'+ req.quote(path_common.fragment)
df = pd.read_html(url)[1]
mill_cities = tuple(df.iloc[:,1])
mill_cities

('Москва',
 'Санкт-Петербург',
 'Новосибирск',
 'Екатеринбург',
 'Казань',
 'Нижний Новгород',
 'Красноярск',
 'Челябинск',
 'Самара',
 'Уфа',
 'Ростов-на-Дону',
 'Краснодар',
 'Омск',
 'Воронеж',
 'Пермь',
 'Волгоград')

In [103]:
# Создадим подзапрос - напишем следующий запрос и сохраним его в переменную temp: объединим таблицы с вакансиями, регионами и 
# работодателями. Применим условие, что наименование организации - Яндекс, а наименование города содержится в кортеже mill_cities. 
# Используем {}, чтобы получить доступ к кортежу через запрос SQL. Выполним группировку по наименованию. Выведем количество вакансий
# count_vacancies. Теперь можно обратиться к этому подзапросу по переменной temp.
# Далее присоединяем строчку 'Total' с общим количеством всех вакансий компании Яндекс (SUM(count_vacancies)) Отсортируем по возрастанию
# count_vacancies.
query_5_7 = f'''WITH temp AS(
                SELECT a.name, COUNT(v.id) count_vacancies
                FROM public.vacancies v
                JOIN public.areas a ON v.area_id = a.id
                JOIN public.employers e ON v.employer_id = e.id
                WHERE e.name = 'Яндекс' AND a.name IN {mill_cities}
                GROUP BY a.name
                )
                SELECT *  
                FROM temp
                UNION ALL
                SELECT 'Total', SUM(count_vacancies)
                FROM temp
                ORDER BY 2
            '''

In [104]:
# Выполним запрос и выведем результат:
df_query_5_7 = pd.read_sql_query(query_5_7, connection)
df_query_5_7


C:\Users\anupa\AppData\Local\Temp\ipykernel_21092\890193919.py:2: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



,name,count_vacancies
0,Омск,21.0
1,Челябинск,22.0
2,Красноярск,23.0
3,Волгоград,24.0
4,Пермь,25.0
5,Казань,25.0
6,Ростов-на-Дону,25.0
7,Самара,26.0
8,Уфа,26.0
9,Краснодар,30.0


***

### Выводы по анализу работодателей
* Лидер по количеству вакансиий - компания Яндекс (1933 вакансии). Это справедливо, поскольку Яндекс -  большая корпорация с более 50 различными веб-службами, которые постоянно развиваются, поэтому всегда нуждаются в новых кадрах. В ТОП-5 также входят крупные холдинги - Ростелеком, Тинькофф, СБЕР, Газпром нефть.
* Также у Яндекса самое большое количество регионов с размещенными вакансиями - 181 шт. Если рассматривать количество вакансий компании в городах-миллионниках, можно убедиться, что больше всего вакансий в Москве (54 шт.). Затем идет Санкт-Петербург (42 шт.), Екатеринбург (39 шт.), Нижний Новгород (36 шт.) и Новосибирск (35 шт.). Мне кажется, в Яндексе много вакансий с удаленной работой, так что местоположение офиса не имеет значение. 
* У 3553 из 23501 работодателей в качестве сферы деятельности указана Разработка программного обеспечения. Данное направление было и остается востребованным, несмотря на колебания IT-рынка в последние годы и внедрение различных новых развивающихся технологий. Поэтому количество организаций, специализующихся на этой, достаточно узкой, сфере, достаточно большое (15% от общего числа). При этом, данные могут быть не полными, поскольку у 8419 фирм не указана сфера деятельности.

# Юнит 6. Предметный анализ

1. Сколько вакансий имеет отношение к данным?

Считаем, что вакансия имеет отношение к данным, если в её названии содержатся слова 'data' или 'данн'.

*Подсказка: Обратите внимание, что названия вакансий могут быть написаны в любом регистре.* 


In [105]:
# Код запроса: с помощью оператора LIKE и % наложим условие - в любом месте названия вакансии должны быть слова 'data' или 'данн'. 
# Посчитаем количество таких вакансий (COUNT(v.id)):
query_6_1 = f'''SELECT COUNT(v.id)
                FROM public.vacancies v
                WHERE lower(v.name) LIKE '%data%' OR lower(v.name) LIKE '%данн%'
            '''

In [106]:
# Выполним запрос и выведем результат:
df_query_6_1 = pd.read_sql_query(query_6_1, connection)
df_query_6_1

C:\Users\anupa\AppData\Local\Temp\ipykernel_21092\2861534987.py:2: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



,count
0,1771


2. Сколько есть подходящих вакансий для начинающего дата-сайентиста? 
Будем считать вакансиями для дата-сайентистов такие, в названии которых есть хотя бы одно из следующих сочетаний:
* 'data scientist'
* 'data science'
* 'исследователь данных'
* 'ML' (здесь не нужно брать вакансии по HTML)
* 'machine learning'
* 'машинн%обучен%'

** В следующих заданиях мы продолжим работать с вакансиями по этому условию.*

Считаем вакансиями для специалистов уровня Junior следующие:
* в названии есть слово 'junior' *или*
* требуемый опыт — Нет опыта *или*
* тип трудоустройства — Стажировка.
 

In [107]:
# Отфильтруем данные с помощью условий с помощью WHERE ... LIKE и %: во первых, в названии вакансии должны быть слова, относящиеся 
# к data science, во вторых - слово 'junior' или в опыте - 'нет опыта' или в типе трудоустройства - 'стажировка'. Поскольку слова 
# в строчках могу быть как заглавными, так и срочными, используем метод lower() для приведения к нижнему регистру. Но для аббревиатуры
# 'ML' не нужно выполнять 'понижение', поскольку в выборку могут попасть ненужные данные (например, TeamLead). 
# Посчитаем количество таких вакансий (COUNT(v.id)):
query_6_2 = f'''SELECT COUNT(v.id)
                FROM public.vacancies v
                WHERE (lower(v.name) LIKE '%data scientist%'
                OR lower(v.name) LIKE '%data science%'
                OR lower(v.name) LIKE '%исследователь данных%'
                OR (v.name LIKE '%ML%' AND lower(v.name) NOT LIKE '%html%')
                OR lower(v.name) LIKE '%machine learning%'
                OR lower(v.name) LIKE '%машинн%обучен%')
                AND (lower(v.name) LIKE '%junior%' 
                OR lower(v.experience) LIKE '%нет опыта%'
                OR lower(v.employment) LIKE '%стажировка%')
            '''

In [108]:
# Выполним запрос и выведем результат:
df_query_6_2 = pd.read_sql_query(query_6_2, connection)
df_query_6_2

C:\Users\anupa\AppData\Local\Temp\ipykernel_21092\3464603721.py:2: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



,count
0,51


3. Сколько есть вакансий для DS, в которых в качестве ключевого навыка указан SQL или postgres?

** Критерии для отнесения вакансии к DS указаны в предыдущем задании.*

In [109]:
# Отфильтруем данные, относящиеся к data science, аналогично предыдущему запросу. Чтобы выполнить поиск без учета регистра любой буквы
# Используем оператор ILIKE (кроме 'ML' - там используем обычный LIKE, как и в прошлый раз). Добавим условие, что v.key_skills содержит
# 'sql' или 'postgres'. Посчитаем количество таких вакансий (COUNT(v.id)):
query_6_3 = f'''SELECT COUNT(v.id)
                FROM public.vacancies v
                WHERE (v.name ILIKE '%data scientist%'
                OR v.name ILIKE '%data science%'
                OR v.name ILIKE '%исследователь данных%'
                OR (v.name LIKE '%ML%' AND v.name NOT ILIKE '%html%')
                OR v.name ILIKE '%machine learning%'
                OR v.name ILIKE '%машинн%обучен%')
                AND (v.key_skills ILIKE '%sql%' 
                OR v.key_skills ILIKE '%postgres%')
            '''

In [110]:
# Выполним запрос и выведем результат:
df_query_6_3 = pd.read_sql_query(query_6_3, connection)
df_query_6_3

C:\Users\anupa\AppData\Local\Temp\ipykernel_21092\1274938648.py:2: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



,count
0,201


4. Проверьте, насколько популярен Python в требованиях работодателей к DS.Для этого вычислите количество вакансий, в которых в качестве ключевого навыка указан Python.

** Это можно сделать помощью запроса, аналогичного предыдущему.*

In [111]:
# Отфильтруем данные, относящиеся к data science, аналогично предыдущему запросу. Добавим условие, что v.key_skills содержит 'python'. 
# Посчитаем количество таких вакансий (COUNT(v.id)):
query_6_4 = f'''SELECT COUNT(v.id)
                FROM public.vacancies v
                WHERE (v.name ILIKE '%data scientist%'
                OR v.name ILIKE '%data science%'
                OR v.name ILIKE '%исследователь данных%'
                OR (v.name LIKE '%ML%' AND v.name NOT ILIKE '%html%')
                OR v.name ILIKE '%machine learning%'
                OR v.name ILIKE '%машинн%обучен%')
                AND v.key_skills ILIKE '%python%' 
            '''

In [112]:
# Выполним запрос и выведем результат:
df_query_6_4 = pd.read_sql_query(query_6_4, connection)
df_query_6_4

C:\Users\anupa\AppData\Local\Temp\ipykernel_21092\1859240553.py:2: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



,count
0,351


5. Сколько ключевых навыков в среднем указывают в вакансиях для DS?
Ответ округлите до двух знаков после точки-разделителя.

In [113]:
# Отфильтруем данные, относящиеся к data science, аналогично предыдущему запросу. Посчитаем среднее значение разниц между длинами 
# значений в столбце с ключевыми навыками length(v.key_skills) и длиной этих же значений без пробелов плюс 1 (length(replace(v.key_skills, 
# CHR(9), '')) + 1). Таким образом как раз получится среднее количество ключевых навыков avg:
query_6_5 = f'''SELECT round(AVG(length(v.key_skills) - length(replace(v.key_skills, CHR(9), '')) + 1), 2) avg
                FROM public.vacancies v
                WHERE v.name ILIKE '%data scientist%'
                OR v.name ILIKE '%data science%'
                OR v.name ILIKE '%исследователь данных%'
                OR (v.name LIKE '%ML%' AND v.name NOT ILIKE '%html%')
                OR v.name ILIKE '%machine learning%'
                OR v.name ILIKE '%машинн%обучен%'
            '''

In [114]:
# Выполним запрос и выведем результат:
df_query_6_5 = pd.read_sql_query(query_6_5, connection)
df_query_6_5

C:\Users\anupa\AppData\Local\Temp\ipykernel_21092\322965811.py:2: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



,avg
0,6.41


6. Напишите запрос, позволяющий вычислить, какую зарплату для DS в **среднем** указывают для каждого типа требуемого опыта (уникальное значение из поля *experience*). 

При решении задачи примите во внимание следующее:
1. Рассматриваем только вакансии, у которых заполнено хотя бы одно из двух полей с зарплатой.
2. Если заполнены оба поля с зарплатой, то считаем зарплату по каждой вакансии как сумму двух полей, делённую на 2. Если заполнено только одно из полей, то его и считаем зарплатой по вакансии.
3. Если в расчётах участвует null, в результате он тоже даст null (посмотрите, что возвращает запрос select 1 + null). Чтобы избежать этой ситуацию, мы воспользуемся функцией [coalesce](https://postgrespro.ru/docs/postgresql/9.5/functions-conditional#functions-coalesce-nvl-ifnull), которая заменит null на значение, которое мы передадим. Например, посмотрите, что возвращает запрос `select 1 + coalesce(null, 0)`

Выясните, на какую зарплату в среднем может рассчитывать дата-сайентист с опытом работы от 3 до 6 лет. Результат округлите до целого числа. 

In [115]:
# Отфильтруем данные, относящиеся к data science, аналогично предыдущему запросу. Добавим условие, что хотя бы одно из значений с зарплатой
# не пустые (v.salary_from IS NOT NULL OR v.salary_to IS NOT NULL). Сгруппируем выборку по опыту работы GROUP BY v.experience. Выведем
# округленное среднее значение заработной платы с помощью функции coalesce - она вычисляет аргументы по порядку и возвращает текущее 
# значение первого выражения, изначально не вычисленного как NULL:
query_6_6 = f'''SELECT DISTINCT v.experience, ROUND(AVG(COALESCE((v.salary_from + v.salary_to)/2, v.salary_from, v.salary_to)))
                FROM public.vacancies v
                WHERE (v.name ILIKE '%data scientist%'
                OR v.name ILIKE '%data science%'
                OR v.name ILIKE '%исследователь данных%'
                OR (v.name LIKE '%ML%' AND v.name NOT ILIKE '%html%')
                OR v.name ILIKE '%machine learning%'
                OR v.name ILIKE '%машинн%обучен%')
                AND (v.salary_from IS NOT NULL OR v.salary_to IS NOT NULL)
                GROUP BY v.experience
            '''

In [116]:
# Выполним запрос и выведем результат:
df_query_6_6 = pd.read_sql_query(query_6_6, connection)
df_query_6_6

C:\Users\anupa\AppData\Local\Temp\ipykernel_21092\3292999428.py:2: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



,experience,round
0,Нет опыта,74643.0
1,От 1 года до 3 лет,139675.0
2,От 3 до 6 лет,243115.0


***

### Выводы по предметному анализу
* Средняя зарплата, на которую может претендовать дата-сайентист, выше общей средней ЗП по России: на вакансии без опыта она составляет	74 643 руб., при опыте от 1 года до 3 лет - 139 675 руб., от 3 до 6 лет - 243 115 руб. Можно заметить, что ЗП по категориям опыта работы отличается практически в два раза.
* В среднем работадатель указывает около 6 (6.41) ключевых навыков для направления data science. При этом у 351 вакансий в качестве ключевого навыка указан python, у 201 - sql или postgres. Заметим, что python требуется намного чаще, что может говорить об особой важности этого навыка и необходимости по-максимуму его "прокачивать".
* Количество вакансий для начинающих дата-сайентистов немного - всего 51, так что junior-специалистам будет довольно сложно найти работу. Важно продолжать преобретать практический опыт и навыки, чтобы претендовать на вакансии не только для начинающих, но и для имеющих опыт работы от 1 года и более.

# Общий вывод по проекту

### Общие выводы

* База данных вакансий HeadHunter насчитывает около 50 тыс. вакансий, что является более чем достаточным для анализа и получения репрезентативных результатов.
* Анализ распределения вакансий относительно регионов показал, что в ТОП-5 по количеству входит Москва (5333 вакансии), Санкт-Петербург (2851), Минск (2112), Новосибирск (2006), Алматы (1892). 
* Средняя нижняя граница зарплатной вилки по всем вакансиям составляет 71 065 руб., верхняя - 110 537 руб. 
* Большинство вакансий (35 367 шт.) предусматривают полный рабочий день и полную занятость, что является самым распространенным графиком работы и типом трудоустройства. 
* Чуть больше половины всех вакансий ищут соискателей с опытом работы 1-3 года.
* Лидер по количеству вакансиий - компания Яндекс (1933 вакансии). Это справедливо, поскольку Яндекс -  большая корпорация с более 50 различными веб-службами.
* Что касается вакансий дата-сайентистов, можно заметить, что ЗП по категориям опыта работы отличается практически в два раза: без опыта - 74 643 руб., опыт работы 1-3 года - 139 675 руб., 3-6 лет - 243 115 руб.
* В среднем работадатель указывает около 6 ключевых навыков для направления data science. Самые важные - владение python, sql или postgres.
* Количество вакансий для начинающих дата-сайентистов немного - всего 51, так что поиск работы для junior-специалиста будет нелегким.  

*Таким образом, рынок вакансий data science, как и любой другой рынок профессий, специфичный и имеет ряд особенностей. Выполняя качественный анализ, можно найти закономерности и выделить важные факторы. Полученные данные помогут настроить рекомендательные системы для кадровых агенств, которые планируют нанять дата-сайентистов.*

### Дополнительные исследования данных
Поскольку основная цель исследования - выполнить анализ данных для настройки рекомендательной системы вакансий на позицию Data Scientist, дополнительные исследования будут выполнены с уже отобранными для таких целей данными.

In [118]:
# Отфильтруем данные, относящиеся к data science, аналогично предыдущим запросам.
query_1 = f'''SELECT *
                FROM public.vacancies v
                WHERE v.name ILIKE '%data scientist%'
                OR v.name ILIKE '%data science%'
                OR v.name ILIKE '%исследователь данных%'
                OR (v.name LIKE '%ML%' AND v.name NOT ILIKE '%html%')
                OR v.name ILIKE '%machine learning%'
                OR v.name ILIKE '%машинн%обучен%'
            '''

In [119]:
# Выполним запрос и выведем результат:
df_query_1 = pd.read_sql_query(query_1, connection)
df_query_1

C:\Users\anupa\AppData\Local\Temp\ipykernel_21092\3760285856.py:2: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



,id,name,key_skills,schedule,experience,employment,salary_from,salary_to,area_id,employer_id
0,50533416,Junior+ CV/ML developer,Python\tSQL\tРабота в команде\tOpenCV\tML,Гибкий график,От 1 года до 3 лет,Полная занятость,NaN,NaN,4,80190
1,54824572,Data Scientist,Oracle Pl/SQL\tБазы данных\tMachine Learning\t...,Полный день,От 1 года до 3 лет,Полная занятость,NaN,NaN,160,111304
2,54825318,Data Scientist (Junior+),Python\tOpenCV\tTensorflow\tComputer Vision\tН...,Полный день,От 1 года до 3 лет,Полная занятость,50000.0,70000.0,72,215942
3,66120847,AI/ML Senior Engineer,Python\tFlask\tLinux\tAWS\tGit,Гибкий график,От 3 до 6 лет,Полная занятость,NaN,NaN,2759,204511
4,55351147,Data Scientist,Python\tSQL\tNLP\tScikit-learn\tMachine Learning,Полный день,От 1 года до 3 лет,Полная занятость,NaN,NaN,53,3127
...,...,...,...,...,...,...,...,...,...,...
475,54918116,Специалист ML/DS,Python\tNumpy\tPyTorch\tPandas\tМатематический...,Полный день,От 3 до 6 лет,Полная занятость,61024.0,73228.0,160,5969955
476,55185186,Data Scientist в области рекомендательных систем,Python\tScikit-learn\tPandas\tGit\tNumpy,Полный день,От 3 до 6 лет,Полная занятость,NaN,NaN,66,3529
477,66106803,C++ разработчик (QML),Qt\tООП\tC++\tMS Visual Studio\tРазработка ком...,Гибкий график,От 3 до 6 лет,Полная занятость,NaN,NaN,54,5966150
478,66125447,Middle+/Senior Data Scientist,SCALA\tSQL\tМатематический анализ\tСтатистичес...,Полный день,От 3 до 6 лет,Полная занятость,NaN,NaN,160,135317


In [120]:
# Ранее для некоторых ключевых навыков было выведено количество вакансий, где они были указаны. Этих ключевых навыков на самом деле
# намного больше. В качестве дополнительного исследования предлагается вывести график соотношения количества вакансий, где указаны
# самые популярные ключевые навыки: 
# Python, SQL, Machine Learning (или ML), Pandas, Математическая статистика, Анализ данных (или) Data analysis, Git, Numpy, NLP.

# Анализ выполним с помощью средств Python, поскольку это удобно и наглядно.

# Для начала создадим объект Series, где преобразуем строки с ключевыми навыками в списки, используя функции .str.split('\t').
# P.S. по таблице df_query_1 можно заметить, что в качестве разделителей в столбце key_skills используется \t - управляющий символ 
# таблуляции (tab).
key_skills_list = df_query_1['key_skills'].str.split('\t')

# Теперь преобразуем key_skills_list в отдельные столбцы с ключевыми навыками с помощью метода One Hot Encoding.

# Добавляем столбцы с признаками в датафрейм df_query_1, применяя lambda-функцию к строкам таблицы key_skills_list: присваиваем вакансии 
# значение 1, если в ней содержится нужный нам ключевой навык, в противном случае 0. Строчкам, где не указаны ключевые навыки (значение 
# None), также присваиваем 0:
df_query_1['Python'] = key_skills_list.apply(lambda x: 0 if x is None else (1 if 'Python' in x else 0))
df_query_1['SQL'] = key_skills_list.apply(lambda x: 0 if x is None else (1 if 'SQL' in x else 0))
df_query_1['Machine Learning'] = key_skills_list.apply(lambda x: 0 if x is None else (1 if 'Machine Learning' in x or 'ML' in x else 0))
df_query_1['Pandas'] = key_skills_list.apply(lambda x: 0 if x is None else (1 if 'Pandas' in x else 0))
df_query_1['Математическая статистика'] = key_skills_list.apply(lambda x: 0 if x is None else (1 if 'Математическая статистика' 
                                                                                               in x else 0))
df_query_1['Анализ данных'] = key_skills_list.apply(lambda x: 0 if x is None else (1 if 'Анализ данных' in x or 
                                                                                   'Data analysisML' in x else 0))
df_query_1['Git'] = key_skills_list.apply(lambda x: 0 if x is None else (1 if 'Git' in x else 0))
df_query_1['Numpy'] = key_skills_list.apply(lambda x: 0 if x is None else (1 if 'Numpy' in x else 0))
df_query_1['NLP'] = key_skills_list.apply(lambda x: 0 if x is None else (1 if 'NLP' in x else 0))

# Выведем результат:
df_query_1

,id,name,key_skills,schedule,experience,employment,salary_from,salary_to,area_id,employer_id,Python,SQL,Machine Learning,Pandas,Математическая статистика,Анализ данных,Git,Numpy,NLP
0,50533416,Junior+ CV/ML developer,Python\tSQL\tРабота в команде\tOpenCV\tML,Гибкий график,От 1 года до 3 лет,Полная занятость,NaN,NaN,4,80190,1,1,1,0,0,0,0,0,0
1,54824572,Data Scientist,Oracle Pl/SQL\tБазы данных\tMachine Learning\t...,Полный день,От 1 года до 3 лет,Полная занятость,NaN,NaN,160,111304,1,0,1,0,0,0,1,0,0
2,54825318,Data Scientist (Junior+),Python\tOpenCV\tTensorflow\tComputer Vision\tН...,Полный день,От 1 года до 3 лет,Полная занятость,50000.0,70000.0,72,215942,1,0,0,0,0,0,0,0,0
3,66120847,AI/ML Senior Engineer,Python\tFlask\tLinux\tAWS\tGit,Гибкий график,От 3 до 6 лет,Полная занятость,NaN,NaN,2759,204511,1,0,0,0,0,0,1,0,0
4,55351147,Data Scientist,Python\tSQL\tNLP\tScikit-learn\tMachine Learning,Полный день,От 1 года до 3 лет,Полная занятость,NaN,NaN,53,3127,1,1,1,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
475,54918116,Специалист ML/DS,Python\tNumpy\tPyTorch\tPandas\tМатематический...,Полный день,От 3 до 6 лет,Полная занятость,61024.0,73228.0,160,5969955,1,0,1,1,0,0,0,1,1
476,55185186,Data Scientist в области рекомендательных систем,Python\tScikit-learn\tPandas\tGit\tNumpy,Полный день,От 3 до 6 лет,Полная занятость,NaN,NaN,66,3529,1,0,0,1,0,0,1,1,0
477,66106803,C++ разработчик (QML),Qt\tООП\tC++\tMS Visual Studio\tРазработка ком...,Гибкий график,От 3 до 6 лет,Полная занятость,NaN,NaN,54,5966150,0,0,0,0,0,0,0,0,0
478,66125447,Middle+/Senior Data Scientist,SCALA\tSQL\tМатематический анализ\tСтатистичес...,Полный день,От 3 до 6 лет,Полная занятость,NaN,NaN,160,135317,0,1,0,1,0,0,0,0,0


In [121]:
# Импортируем библиотеку plotly.express для визуализации полученных данных
import plotly.express as px

# Подготовим данные - посчитаем суммы значений новых столбцов и сохраним в таблице Series:
bar_data = df_query_1[['Python', 'SQL', 'Machine Learning', 'Pandas', 'Математическая статистика', 'Анализ данных', 
                       'Git', 'Numpy', 'NLP']].sum()

# Выведем таблицу bar_data для удобства:
display(bar_data)

# Построим столбчатую диаграмму. По оси x выведем названия навыков (индексы таблицы bar_data), а по оси y - значения (количество вакансий).
# "Столбикам" диаграммы присвоим различные цвета для наглядности (color = bar_data.index), в параметре labels зададим названия осей
# и легенды.
fig = px.bar(
    data_frame = bar_data,
    x = bar_data.index,
    y = bar_data,
    color = bar_data.index,
    text = bar_data,
    labels = {'y': 'Количество резюме', 'index': 'Ключевые навыки', 'variable': 'Ключевые навыки'},
    title = 'Популярности основных ключевых навыков в вакансиях data science на HeadHunter'
)

# Выведем полученный график
fig.show()



Python                       348
SQL                          191
Machine Learning             149
Pandas                        52
Математическая статистика     62
Анализ данных                 46
Git                           66
Numpy                         34
NLP                           40
dtype: int64

### Выводы по дополнительному анализу
* Исследование показало, что самым популярным навыком является владение Python (как и можно было догадаться). Можно заметить, что библиотеки Pandas и Numpy выделены отдельно - так делают в некоторых вакансиях, чтобы подчеркнуть важность этих навыков. Но все же они тоже относятся к Python, так что самым показательным является именно он. 
* Второй по значимости - SQL. Это два базовых key skills для data scientist, поэтому даже в курсе SkillFactory изучаются в первую очередь.
* Полученное распределение количества вакансий по ключевым навыкам может помочь junior-специалистам понять, на чем делать акцент в обучении, и какие key skills чаще всего требуются на практике.

In [122]:
# По окончанию работы закрываем соединение
connection.close()